In [19]:
%pip install -U pip setuptools wheel
%pip install -U spacy
%python -m spacy download en_core_web_sm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).


In [20]:
%pip install nest_asyncio fastapi uvicorn
%pip install -U pip setuptools wheel
%pip install -U spacy
!python -m spacy download en_core_web_sm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
OMP: Error #15: Initializing libiomp5.dylib, but found libomp.dylib already initialized.
OMP: Hint This means that multiple copies of the OpenMP runtime have been linked into the program. That is dangerous, since it can degrade performance or cause incorrect results. The best thing to do is to ensure that only a single OpenMP runtime is linked into the process, e.g. by avoiding static linking of the OpenMP runtime in any library. As an unsafe, unsupported, undocumented workaround you can set the environment variable KMP_DUPLICATE_LIB_OK=TRUE to allow the program to continue to execute, but that may cause crashes or silently produce incorrect results. For more information, please see http://www.intel.com/software/products/support/.


In [21]:
%pip install openai
%pip install pandas
%pip install numpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [22]:
import pickle

with open('distances.pkl', 'rb') as f:
    distances = pickle.load(f)

In [23]:
# load environment variables
import dotenv
dotenv.load_dotenv()
import os
import openai
openai.organization = "org-9bUDqwqHW2Peg4u47Psf9uUo"
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.Model.list()

<OpenAIObject list at 0x7fb5119d0270> JSON: {
  "data": [
    {
      "created": 1649358449,
      "id": "babbage",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampling": true,
          "allow_search_indices": false,
          "allow_view": true,
          "created": 1669085501,
          "group": null,
          "id": "modelperm-49FUp5v084tBB49tC4z8LPH5",
          "is_blocking": false,
          "object": "model_permission",
          "organization": "*"
        }
      ],
      "root": "babbage"
    },
    {
      "created": 1649359874,
      "id": "davinci",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sa

In [24]:
question_w_outline = '''
Write a short essay given this outline:
• It took me eighteen years to realize what an extraordinary influence my mother has been on my life
• Her mother's enthusiasm for learning
• Learning through travel by using the example of a trip to Greece
• While I treasure the various worlds my mother has opened to me abroad, my life has been equally transformed by what she has shown me just two miles from my house
• Her mother's dedication to the community
• Her multiple volunteer activities such as helping at the local soup kitchen
• Everything that my mother has ever done has been overshadowed by the thought behind it
• She has enriched my life with her passion for learning, and changed it with her devotion to humanity
• Next year, I will find a new home miles away. However, my mother will always be by my side

'''

In [25]:
sentences = [
    'It took me eighteen years to realize what an extraordinary influence my mother has been on my life.', 
    "She's the kind of person who has thoughtful discussions about which artist she would most want to have her portrait painted by (Sargent), the kind of mother who always has time for her four children, and the kind of community leader who has a seat on the board of every major project to assist Washington's impoverished citizens.Growing up with such a strong role model, I developed many of her enthusiasms.I not only came to love the excitement of learning simply for the sake of knowing something new, but I also came to understand the idea of giving back to the community in exchange for a new sense of life, love, and spirit.", 
    "My mother's enthusiasm for learning is most apparent in travel.", 
    'Despite the fact that we were traveling with fourteen-month-old twins, we managed to be at each ruin when the site opened at sunrise.', 
    'I vividly remember standing in an empty amphitheatre pretending to be an ancient tragedian, picking out my favorite sculpture in the Acropolis museum, and inserting our family into modified tales of the battle at Troy.', 'I was nine years old when my family visited Greece.', 
    'Every night for three weeks before the trip, my older brother Peter and I sat with my mother on her bed reading Greek myths and taking notes on the Greek Gods.', 
    'Eight years and half a dozen passport stamps later I have come to value what I have learned on these journeys about global history, politics and culture, as well as my family and myself.', 
    "While I treasure the various worlds my mother has opened to me abroad, my life has been equally transformed by what she has shown me just two miles from my house.As a ten year old, I often accompanied my mother to (name deleted), a local soup kitchen and children's center.", 
    'While she attended meetings, I helped with the Summer Program by chasing children around the building and performing magic tricks.Having finally perfected the "floating paintbrush" trick, I began work as a full time volunteer with the five and six year old children last June.It is here that I met Jane Doe, an exceptionally strong girl with a vigor that is contagious.', 
    "At the end of the summer, I decided to continue my work at (name deleted) as Jane's tutor.Although the position is often difficult, the personal rewards are beyond articulation.", 
    'In the seven years since I first walked through the doors of (name deleted), I have learned not only the idea of giving to others, but also of deriving from them a sense of spirit.', 
    'Everything that my mother has ever done has been overshadowed by the thought behind it.While the raw experiences I have had at home and abroad have been spectacular, I have learned to truly value them by watching my mother.She has enriched my life with her passion for learning, and changed it with her devotion to humanity.In her endless love of everything and everyone she is touched by, I have seen a hope and life that is truly exceptional.Next year, I will find a new home miles away.', 
    'However, my mother will always be by my side.'
]

In [26]:
def all_unique_pos(lst, element):
    res = []
    for i in range(len(lst)):
        new_list = lst[:]
        new_list.pop(lst.index(element))
        res.append(new_list[:i] + [element] + new_list[i:])
    return res

In [27]:
def full_request_template(question, sents):
    return question + ' '.join(sents)

In [28]:
import joblib
from joblib import Memory

In [29]:
memory = Memory("./joblib_cache", verbose = 0)

@memory.cache
def get_response(full_request):
  response = openai.Completion.create(
      model="text-davinci-003",
      prompt = full_request,
      temperature=0.7,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      logprobs=10,
      echo=True
    )
  return response

In [30]:
def get_all_responses(options):
    response = []
    for x in options:
        response.append(get_response(full_request_template(question_w_outline, x)))
    return response

In [31]:
result = get_all_responses(all_unique_pos(sentences, 'It took me eighteen years to realize what an extraordinary influence my mother has been on my life.'))
#result

In [32]:
def compute_log_probs(question_w_outline, original_text, response):
    # 1. Start point: text offset of the first word from our original essay
    start_point = len(question_w_outline)   
    # 2. Start index
    start_index = response.choices[0].logprobs.text_offset.index(start_point)
    # 3. Length of the original text
    len_original_text = len(original_text)    
    # 4. End point = Start point + Length of the original text
    end_point = start_point + len_original_text - 1
    # 5. end index
    end_index = response.choices[0].logprobs.text_offset.index(end_point)
    # total logprobs
    total = 0
    for x in range(start_index, end_index + 1):
        total = total + response.choices[0].logprobs.token_logprobs[x]
    return total    

In [33]:
combinations = all_unique_pos(sentences, 'It took me eighteen years to realize what an extraordinary influence my mother has been on my life.')
#combinations

In [34]:
# For all the possibilities
def all_the_log_probs():
    for x in sentences:
        all_unique_pos(sentences, x)
        print(x)

In [35]:
logprobs = []
for i in range(len(combinations)):
    original_text = ' '.join(combinations[i])
    logprobs.append(compute_log_probs(question_w_outline, original_text, result[i]))
    
#logprobs

In [36]:
from typing import Optional 
from fastapi import FastAPI, Response
from fastapi.middleware.cors import CORSMiddleware 
from fastapi.responses import HTMLResponse, PlainTextResponse
from fastapi.staticfiles import StaticFiles
import json


app = FastAPI() 

origins = [ 
    "http://localhost", 
    "http://localhost:8080", 
] 

app.add_middleware( 
    CORSMiddleware, 
    allow_origins=origins, 
    allow_credentials=True, 
    allow_methods=["*"], 
    allow_headers=["*"], 
) 

@app.get("/", response_class=HTMLResponse) 
def read_root(): 
    return open("index.html").read()

app.mount("/styles", StaticFiles(directory="styles"), name="styles")

@app.get("/items/{item_id}") 
def read_item(item_id: int, q: Optional[str] = None): 
    return {"item_id": item_id, "q": q}

@app.get("/item/match")
async def read_items():
    data = (distances)
    return data

@app.get("/item/logprobs")
async def read_items():
    data = (
        logprobs
    )
    return data
    

    

@app.get("/item")
async def read_items():
    # data = "My mother's enthusiasm for learning is most apparent in travel. Despite the fact that we were traveling with fourteen-month-old twins, we managed to be at each ruin when the site opened at sunrise. I vividly remember standing in an empty amphitheatre pretending to be an ancient tragedian, picking out my favorite sculpture in the Acropolis museum, and inserting our family into modified tales of the battle at Troy. I was nine years old when my family visited Greece. Every night for three weeks before the trip, my older brother Peter and I sat with my mother on her bed reading Greek myths and taking notes on the Greek Gods. Eight years and half a dozen passport stamps later I have come to value what I have learned on these journes about global history, politics and culture, as well as my family and myself."

    data = (
            {
                "text":"I vividly remember standing in an empty amphitheatre pretending to be an ancient tragedian, picking out my favorite sculpture in the Acropolis museum, and inserting our family into modified tales of the battle at Troy. ",
                "comment":"",
                "new-order":"0"
                
            },
            {
                "text":"I was nine years old when my family visited Greece. ",
                "comment":"",
                "new-order":"2"
                
            },
            {
                "text":"Every night for three weeks before the trip, my older brother Peter and I sat with my mother on her bed reading Greek myths and taking notes on the Greek Gods. ",
                "comment":"",
                "new-order":"3"
                
            },
            {
                "text":"Eight years and half a dozen passport stamps later I have come to value what I have learned on these journes about global history, politics and culture, as well as my family and myself. ",
                "comment":"",
                "new-order":"1"
            }
    )
    return data

import nest_asyncio 
import uvicorn 

nest_asyncio.apply() 
uvicorn.run(app, port=8000) 

INFO:     Started server process [22530]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:62951 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:62951 - "GET /item/match HTTP/1.1" 200 OK
